In [1]:
pip install python-binance


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 13.9 MB/s eta 0:00:00


In [2]:
from binance.client import Client
import pandas as pd
import numpy as np

# Initialize the client
api_key = 'y5kNTgiGwQ0h2I1lKr6Exvf0CE8lXdT5a1t3ElemCIvu0eSpCFGPsDy8Kr2I3jtE'       # Replace with your Binance API key
api_secret = 'yr9SWJ1eyXbIubpuC8YfiTFI70sivCnSTmPY3Qtb4E5XuuJ89P8w6NX4arwcgbQR' # Replace with your Binance API secret
client = Client(api_key, api_secret)

# Fetch historical klines (candlestick data)
# Example: Fetch daily data for BTC/USDT from January 1, 2017, to present
klines = client.get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_1DAY, "1 Jan, 2017")

# Convert to DataFrame
data = pd.DataFrame(klines, columns=[
    'Open Time', 'Open', 'High', 'Low', 'Close', 'Volume',
    'Close Time', 'Quote Asset Volume', 'Number of Trades',
    'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume', 'Ignore'
])

# Convert time columns to datetime
data['Date'] = pd.to_datetime(data['Open Time'], unit='ms')
data['Close'] = data['Close'].astype(float)
data = data[['Date', 'Close']]


In [3]:
data


,Date,Close
0,2017-08-17,4285.08
1,2017-08-18,4108.37
2,2017-08-19,4139.98
3,2017-08-20,4086.29
4,2017-08-21,4016.00
...,...,...
2645,2024-11-13,90375.20
2646,2024-11-14,87325.59
2647,2024-11-15,91032.07
2648,2024-11-16,90586.92


In [4]:
from sklearn.preprocessing import MinMaxScaler

# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

# Create sequences
def create_sequences(data, timesteps):
    X, y = [], []
    for i in range(timesteps, len(data)):
        X.append(data[i - timesteps:i])
        y.append(data[i])
    return np.array(X), np.array(y)

best_timestep = 30
X, y = create_sequences(scaled_data, best_timestep)
X = X.reshape((X.shape[0], X.shape[1], 1))


In [5]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

# Build the model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(best_timestep, 1)),
    LSTM(50),
    Dense(1)
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model
model.fit(X, y, epochs=50, batch_size=32, verbose=1)

# Save the model and scaler
model.save('lstm_binance_model.h5')
joblib.dump(scaler, 'scaler_binance.pkl')


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


82/82 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - loss: 0.0217
Epoch 2/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - loss: 9.1913e-04
Epoch 3/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - loss: 7.7215e-04
Epoch 4/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 7.7659e-04
Epoch 5/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - loss: 7.1365e-04
Epoch 6/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 5.6680e-04
Epoch 7/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - loss: 6.4185e-04
Epoch 8/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 5.2022e-04
Epoch 9/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - loss: 4.1904e-04
Epoch 10/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 4.2427e-04
Epoch 11/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 3.9770e-04
Epoch 12/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - loss: 3.7897e-04
Epoch 13/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - loss: 4.0211e-04
Epoch 14/50
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 3.5005e-04
Epoch 15/50
82/82 ━━━━━━━━━━━━

NameError: name 'joblib' is not defined

In [6]:
!pip install joblib

import joblib

In [7]:
# Save the model and scaler
model.save('lstm_binance_model.h5')
joblib.dump(scaler, 'scaler_binance.pkl')

['scaler_binance.pkl']

In [8]:
def predict_future(model, last_sequence, scaler, target_date, last_known_date):
    target_date = pd.to_datetime(target_date)
    last_known_date = pd.to_datetime(last_known_date)
    days_ahead = (target_date - last_known_date).days
    if days_ahead <= 0:
        raise ValueError("Target date must be in the future compared to the last known date.")

    current_input = last_sequence.copy()
    prediction = None
    for _ in range(days_ahead):
        prediction = model.predict(current_input.reshape(1, best_timestep, 1))
        current_input = np.append(current_input[1:], prediction, axis=0)

    predicted_price = scaler.inverse_transform(prediction)[0, 0]
    return predicted_price

# Get the last sequence for prediction
last_sequence = scaled_data[-best_timestep:]
last_known_date = data['Date'].iloc[-1]
user_date = '2024-12-04'  # Example target date

predicted_price = predict_future(model, last_sequence, scaler, user_date, last_known_date)
print(f"Predicted Bitcoin price for {user_date}: ${predicted_price:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Predicted Bitcoin price for 2024-12-04: $89057.77


In [9]:
def predict_future(model, last_sequence, scaler, target_date, last_known_date):
    target_date = pd.to_datetime(target_date)
    last_known_date = pd.to_datetime(last_known_date)
    days_ahead = (target_date - last_known_date).days
    if days_ahead <= 0:
        raise ValueError("Target date must be in the future compared to the last known date.")

    current_input = last_sequence.copy()
    prediction = None
    for _ in range(days_ahead):
        prediction = model.predict(current_input.reshape(1, best_timestep, 1))
        current_input = np.append(current_input[1:], prediction, axis=0)

    predicted_price = scaler.inverse_transform(prediction)[0, 0]
    return predicted_price

# Get the last sequence for prediction
last_sequence = scaled_data[-best_timestep:]
last_known_date = data['Date'].iloc[-1]
user_date = '2024-11-18'  # Example target date

predicted_price = predict_future(model, last_sequence, scaler, user_date, last_known_date)
print(f"Predicted Bitcoin price for {user_date}: ${predicted_price:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Predicted Bitcoin price for 2024-11-18: $89690.87
